<a href="https://colab.research.google.com/github/timgluz/colab_notebooks/blob/master/Cifar10_liveproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Dataset

Keras provides the CIFAR dataset for us to load using load_data() method. 

In [ ]:
import keras
from keras.datasets import cifar10
 
# load the pre-shuffled train and test data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
 
fig = plt.figure(figsize=(20,5))
for i in range(36):
    ax = fig.add_subplot(3, 12, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_train[i]))

# Image preprocessing

**TIP** When using gradient descent, you should ensure that all features have a similar scale or else it will take much longer to converge.

In [ ]:
# rescale images by dividing the pixel values by 255 [0,255] --> [0,1]
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [ ]:
# break training set into training and validation sets
(x_train, x_valid) = x_train[5000:], x_train[:5000]
(y_train, y_valid) = y_train[5000:], y_train[:5000]
 
# print shape of training set
print('x_train shape:', x_train.shape)
 
# print number of training, validation, and test images
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_valid.shape[0], 'validation samples')

In [ ]:
from keras.utils import np_utils
 
# one-hot encode the labels
num_classes = len(np.unique(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [ ]:
# normalize also y_valid
y_valid = keras.utils.to_categorical(y_valid, num_classes)

# Building a model

The network consists of three CONV and POOL layers + 2 FC layers (also called dense). Note that we will use the RELU activation function for all the hidden layers. In the last dense layer we will use a Softmax activation function with 10 nodes to return an array of 10 probability scores (summing to 1). Each score will be the probability that the current image belong to our 10 image classes.

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([

  # layer.1 - specifying the input shape
  Conv2D(filters = 16, kernel_size = 2, padding = 'same', activation = 'relu',
        input_shape = (32, 32, 3)),
  MaxPooling2D(pool_size = 2),

  # layer.2
  Conv2D(filters = 32, kernel_size = 2, padding = 'same', activation = 'relu'),
  MaxPooling2D(pool_size = 2),

  # layer.3
  Conv2D(filters = 64, kernel_size = 2, padding = 'same', activation = 'relu'),
  MaxPooling2D(pool_size = 2),

  # layer.4 - dropout layer to avoid overfitting with 30% rate
  Dropout(0.3),

  # flatten the last feature map into a vector of features
  Flatten(),

  # add the first FullyConnected layer
  Dense(500, activation = 'relu'),
  Dropout(0.4),

  # final layer
  Dense(num_classes, activation = 'softmax')
])


In [ ]:
# compile the model
# momentum optimizer gives accuracy 73%
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

optimizer2 = keras.optimizers.RMSprop(lr = 0.0001, rho = 0.9)
model.compile(loss='categorical_crossentropy', optimizer=optimizer2, metrics=['accuracy'])
model.summary()

# Train the model


In [ ]:
from keras.callbacks import ModelCheckpoint  
 
 
# train the model
checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, save_best_only=True)
 
hist = model.fit(x_train, y_train, batch_size=32, 
                 epochs=100,
                 validation_data=(x_valid, y_valid), 
                 callbacks=[checkpointer], 
                 verbose=2, 
                 shuffle=True)

# Evaluate the model

The last step is to evaluate our model and calculate the accuracy value in percentage of how many times our model is correct in predicting the image classification.

In [ ]:
# load the weights that yielded the best validation accuracy
model.load_weights('model.weights.best.hdf5')

In [ ]:
# evaluate and print test accuracy
score = model.evaluate(x_test, y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])